# Redbull Classification Pipeline

### Reference

https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/official/pipelines/google_cloud_pipeline_components_model_train_upload_deploy.ipynb \
https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/self-paced-labs/vertex-ai/vertex-ai-qwikstart/lab_exercise.ipynb \
https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/official/custom/sdk-custom-image-classification-batch.ipynb 

In [1]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

In [2]:
! pip3 install -U google-cloud-storage $USER_FLAG

In [3]:
! pip3 install $USER kfp google-cloud-pipeline-components --upgrade

### Restart the kernel

In [4]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [1]:
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
! python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

KFP SDK version: 1.8.3
google_cloud_pipeline_components version: 0.1.7


In [1]:
PROJECT_ID = "[your-project-id]"
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

Project ID: uplifted-road-327005


In [2]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


In [3]:
REGION="asia-northeast1" #Tokyo region

In [4]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y-%m-%d-%H-%M%S")
print(TIMESTAMP)

2021-10-02-06-0410


In [5]:
BUCKET_NAME = "gs://pipeline-" + TIMESTAMP

In [6]:
! gsutil mb -l $REGION $BUCKET_NAME

Creating gs://pipeline-2021-10-02-06-0410/...


In [7]:
! gsutil ls -al $BUCKET_NAME

In [8]:
SERVICE_ACCOUNT = "[your-service-account]"

if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your GCP project id from gcloud
    shell_output = !gcloud auth list 2>/dev/null
    SERVICE_ACCOUNT = "567299754976-compute@developer.gserviceaccount.com" #shell_output[2].strip()
    print("Service Account:", SERVICE_ACCOUNT)

Service Account: 567299754976-compute@developer.gserviceaccount.com


In [9]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_NAME

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_NAME

In [10]:
import os
import google.cloud.aiplatform as aip

In [11]:
PIPELINE_ROOT = "{}/pipeline_root/redbull".format(BUCKET_NAME)

In [12]:
import kfp
from google_cloud_pipeline_components import aiplatform as gcc_aip
from kfp.v2.dsl import component
from kfp.v2.google import experimental

### Generate required files

In [13]:
# this is the name of your model subdirectory you will write your model code to. It is already created in your lab directory.
MODEL_NAME="inception-resnetv2"

In [14]:
os.chdir("/home/jupyter/")
os.getcwd()

'/home/jupyter'

In [15]:
!mkdir {MODEL_NAME} && cd {MODEL_NAME} && mkdir trainer

mkdir: cannot create directory ‘inception-resnetv2’: File exists


In [16]:
%%writefile {MODEL_NAME}/trainer/model.py
import os
import logging
import tempfile
from google.cloud import storage
from pathlib import Path
    
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from explainable_ai_sdk.metadata.tf.v2 import SavedModelMetadataBuilder
from tensorflow.python.framework import dtypes
from tensorflow.keras.preprocessing import image_dataset_from_directory

BATCH_SIZE = 4
IMG_SIZE = (160, 160)
IMG_SHAPE = IMG_SIZE + (3,)

def create_dataset(bucket_name, prefix):
    
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=prefix)

    for blob in blobs:
        if blob.name.endswith("/"):
            continue
        file_split = blob.name.split("/")
        directory = "/".join(file_split[0:-1])
        Path(directory).mkdir(parents=True, exist_ok=True)
        blob.download_to_filename(blob.name)   
    
    train_dir = os.path.join(prefix, 'train')
    validation_dir = os.path.join(prefix, 'validation')

    
    train_dataset = image_dataset_from_directory(train_dir,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)
    
    validation_dataset = image_dataset_from_directory(validation_dir,
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE,
                                                  image_size=IMG_SIZE)
    
    val_batches = tf.data.experimental.cardinality(validation_dataset)
    test_dataset = validation_dataset.take(val_batches // 4)
    validation_dataset = validation_dataset.skip(val_batches // 4)
    
    print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
    print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

    # Configure the dataset for performance
    AUTOTUNE = tf.data.experimental.AUTOTUNE

    train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
    validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
    test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)
    
    return train_dataset, validation_dataset, test_dataset


def build_model(hparams):
    """Build and compile a TensorFlow Keras InceptionResNetV2."""
    
    # Data Augmentation
    data_augmentation = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
      tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    ])
    
    # Data Preprocessing
    preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
    
    # Base Model
    base_model = tf.keras.applications.InceptionResNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
    
    base_model.trainable = False #Free the convolutional base
    
    # Classification head
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

    # Prediction Layer
    prediction_layer = tf.keras.layers.Dense(1)
    
    
    # Build by Keras Functional API
    inputs = tf.keras.Input(shape=(160, 160, 3))
    x = data_augmentation(inputs)
    x = preprocess_input(x)
    x = base_model(x, training=False)
    x = global_average_layer(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs, outputs)
    
    # Compile the model
    base_learning_rate = 0.0001
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    
    return model


def train_evaluate_explain_model(hparams):
    """Train, evaluate, explain TensorFlow Keras DNN Regressor.
    Args:
      hparams(dict): A dictionary containing model training arguments.
    Returns:
      history(tf.keras.callbacks.History): Keras callback that records training event history.
    """
    
    train_dataset, validation_dataset, test_dataset = create_dataset(hparams['bucket_name'], 
                                                                     hparams['prefix'])
    
    model = build_model(hparams)
    logging.info(model.summary())
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=hparams['tensorboard-dir'],
        histogram_freq=1)
    
    # Reduce overfitting and shorten training times.
    earlystopping_callback = tf.keras.callbacks.EarlyStopping(patience=2)
    
    # Ensure your training job's resilience to VM restarts.
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath= hparams['checkpoint-dir'],
        save_weights_only=True,
        monitor='val_loss',
        mode='min')
    
    # Virtual epochs design pattern:
    # https://medium.com/google-cloud/ml-design-pattern-3-virtual-epochs-f842296de730
    TOTAL_TRAIN_EXAMPLES = int(hparams['stop-point'] * hparams['n-train-examples'])
    STEPS_PER_EPOCH = (TOTAL_TRAIN_EXAMPLES // (hparams['batch-size']*hparams['n-checkpoints']))    
    
    
    history = model.fit(train_dataset,
                    validation_data=validation_dataset,
                    steps_per_epoch=STEPS_PER_EPOCH,
                    epochs=hparams['n-checkpoints'],
                    callbacks=[[tensorboard_callback,
                                    earlystopping_callback,
                                    checkpoint_callback]])
    
    
    logging.info(model.evaluate(test_dataset))
    
    # Create a temp directory to save intermediate TF SavedModel prior to Explainable metadata creation.
    tmpdir = tempfile.mkdtemp()
    
    # Export Keras model in TensorFlow SavedModel format.
    model.save(tmpdir)
    
    # Annotate and save TensorFlow SavedModel with Explainable metadata to GCS.
    builder = SavedModelMetadataBuilder(tmpdir)
    builder.save_model_with_metadata(hparams['model-dir'])
    
    return history

Overwriting inception-resnetv2/trainer/model.py


### `task.py` as an entrypoint to the custom ML model container

In [17]:
%%writefile {MODEL_NAME}/trainer/task.py
import os
import argparse

from trainer import model

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    # Vertex custom container training args. These are set by Vertex AI during training but can also be overwritten.
    parser.add_argument('--model-dir', dest='model-dir',
                        default=os.environ['AIP_MODEL_DIR'], type=str, help='Model dir.')
    parser.add_argument('--checkpoint-dir', dest='checkpoint-dir',
                        default=os.environ['AIP_CHECKPOINT_DIR'], type=str, help='Checkpoint dir set during Vertex AI training.')    
    parser.add_argument('--tensorboard-dir', dest='tensorboard-dir',
                        default=os.environ['AIP_TENSORBOARD_LOG_DIR'], type=str, help='Tensorboard dir set during Vertex AI training.')    
    parser.add_argument('--data-format', dest='data-format',
                        default=os.environ['AIP_DATA_FORMAT'], type=str, help="Tabular data format set during Vertex AI training. E.g.'csv', 'bigquery'")
    parser.add_argument('--training-data-uri', dest='training-data-uri',
                        default=os.environ['AIP_TRAINING_DATA_URI'], type=str, help='Training data GCS or BQ URI set during Vertex AI training.')
    parser.add_argument('--validation-data-uri', dest='validation-data-uri',
                        default=os.environ['AIP_VALIDATION_DATA_URI'], type=str, help='Validation data GCS or BQ URI set during Vertex AI training.')
    parser.add_argument('--test-data-uri', dest='test-data-uri',
                        default=os.environ['AIP_TEST_DATA_URI'], type=str, help='Test data GCS or BQ URI set during Vertex AI training.')
    # Model training args.
    parser.add_argument('--learning-rate', dest='learning-rate', default=0.001, type=float, help='Learning rate for optimizer.')
    parser.add_argument('--dropout', dest='dropout', default=0.2, type=float, help='Float percentage of DNN nodes [0,1] to drop for regularization.')    
    parser.add_argument('--batch-size', dest='batch-size', default=16, type=int, help='Number of examples during each training iteration.')    
    parser.add_argument('--n-train-examples', dest='n-train-examples', default=2638, type=int, help='Number of examples to train on.')
    parser.add_argument('--stop-point', dest='stop-point', default=10, type=int, help='Number of passes through the dataset during training to achieve convergence.')
    parser.add_argument('--n-checkpoints', dest='n-checkpoints', default=10, type=int, help='Number of model checkpoints to save during training.')
    
    # Custom arguments
    parser.add_argument('--bucket_name', dest='bucket_name',  default='image-classification-datasets', type=str)
    parser.add_argument('--prefix', dest='prefix',  default='redbull', type=str)
    
    args = parser.parse_args()
    hparams = args.__dict__

    model.train_evaluate_explain_model(hparams)

Overwriting inception-resnetv2/trainer/task.py


### Write a `Dockerfile` for the custom ML model container

In [18]:
%%writefile {MODEL_NAME}/Dockerfile
# Specifies base image and tag.
# https://cloud.google.com/vertex-ai/docs/general/deep-learning
# https://cloud.google.com/deep-learning-containers/docs/choosing-container
FROM gcr.io/deeplearning-platform-release/tf2-cpu.2-3

# Sets the container working directory.
WORKDIR /root

# Copies the requirements.txt into the container to reduce network calls.
COPY requirements.txt .
# Installs additional packages.
RUN pip3 install -U -r requirements.txt

# Copies the trainer code to the docker image.
COPY . /trainer

# Sets the container working directory.
WORKDIR /trainer

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

Overwriting inception-resnetv2/Dockerfile


In [19]:
%%writefile {MODEL_NAME}/requirements.txt
explainable-ai-sdk==1.3.0
tensorflow-io==0.15.0
pyarrow

Overwriting inception-resnetv2/requirements.txt


### Create Artifact Repository

In [20]:
ARTIFACT_REPOSITORY="redbull-pipeline"

In [21]:
# Create an Artifact Repository using the gcloud CLI.
!gcloud artifacts repositories create $ARTIFACT_REPOSITORY \
--repository-format=docker \
--location=$REGION \
--description="Artifact registry for ML custom training images for Redbull Classification"

ERROR: (gcloud.artifacts.repositories.create) ALREADY_EXISTS: the repository already exists


In [22]:
IMAGE_NAME="inceptionresnet-pipeline"
IMAGE_TAG="latest"
IMAGE_URI=f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{ARTIFACT_REPOSITORY}/{IMAGE_NAME}:{IMAGE_TAG}"

In [23]:
IMAGE_URI

'asia-northeast1-docker.pkg.dev/uplifted-road-327005/redbull-pipeline/inceptionresnet-pipeline:latest'

### Create `cloudbuild.yaml` instructions

In [24]:
cloudbuild_yaml = f"""steps:
- name: 'gcr.io/cloud-builders/docker'
  args: [ 'build', '-t', '{IMAGE_URI}', '.' ]
images: 
- '{IMAGE_URI}'"""

with open(f"{MODEL_NAME}/cloudbuild.yaml", "w") as fp:
    fp.write(cloudbuild_yaml)

### Build and submit the container image to Artifact Repository

Image name must be the lowercase

In [25]:
!gcloud builds submit --timeout=20m --config {MODEL_NAME}/cloudbuild.yaml {MODEL_NAME}

Creating temporary tarball archive of 5 file(s) totalling 9.8 KiB before compression.
Uploading tarball of [inception-resnetv2] to [gs://uplifted-road-327005_cloudbuild/source/1633154662.384953-173856a20cb64af39f3cbed6fdef75ab.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/uplifted-road-327005/locations/global/builds/91a20f73-c54e-42d9-bc27-66f22ba5f4bd].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/91a20f73-c54e-42d9-bc27-66f22ba5f4bd?project=567299754976].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "91a20f73-c54e-42d9-bc27-66f22ba5f4bd"

FETCHSOURCE
Fetching storage object: gs://uplifted-road-327005_cloudbuild/source/1633154662.384953-173856a20cb64af39f3cbed6fdef75ab.tgz#1633154662700964
Copying gs://uplifted-road-327005_cloudbuild/source/1633154662.384953-173856a20cb64af39f3cbed6fdef75ab.tgz#1633154662700964...
/ [1 files][  3.4 KiB/  3.4 KiB]                                              

### Initialize Vertex SDK for Python

In [26]:
BASE_OUTPUT_DIR= f"gs://{BUCKET_NAME}/vertex-custom-training-{MODEL_NAME}-{TIMESTAMP}"

In [27]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

In [28]:
@component
def initiate(input1: str):
    print("training task: {}".format(input1))

In [29]:
#hp_dict: str = '{"num_hidden_layers": 3, "hidden_size": 32, "learning_rate": 0.01, "epochs": 1, "steps_per_epoch": -1}'
#data_dir: str = "gs://aju-dev-demos-codelabs/bikes_weather/"
#TRAINER_ARGS = ["--data-dir", data_dir, "--hptune-dict", hp_dict]

# create working dir to pass to job spec
WORKING_DIR = f"{PIPELINE_ROOT}/{TIMESTAMP}"


MODEL_DISPLAY_NAME = f"train_deploy{TIMESTAMP}"
#print(TRAINER_ARGS, WORKING_DIR, MODEL_DISPLAY_NAME)


@kfp.dsl.pipeline(name="train-endpoint-deploy" + TIMESTAMP)
def pipeline(
    project: str = PROJECT_ID,
    model_display_name: str = MODEL_DISPLAY_NAME,
    serving_container_image_uri: str = "us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-3:latest",
):

    train_task = initiate("model training")
    # https://github.com/kubeflow/pipelines/blob/master/sdk/python/kfp/v2/google/experimental/custom_job.py
    experimental.run_as_aiplatform_custom_job(
        train_task,
        worker_pool_specs=[
            {
                "containerSpec": {
                    "args": [],
                    "env": [{"name": "AIP_MODEL_DIR",          "value": WORKING_DIR},
                            {"name": "AIP_CHECKPOINT_DIR",     "value": WORKING_DIR},
                            {"name": "AIP_TENSORBOARD_LOG_DIR","value": WORKING_DIR},
                            {"name": "AIP_DATA_FORMAT",        "value": "csv"},
                            {"name": "AIP_TRAINING_DATA_URI",  "value": "uri"},
                            {"name": "AIP_VALIDATION_DATA_URI","value": "uri"},
                            {"name": "AIP_TEST_DATA_URI",      "value": "uri"}],
                    "imageUri": IMAGE_URI,
                },
                "replicaCount": "1",
                "machineSpec": {
                    "machineType": "n1-standard-16",
                    #"accelerator_type": aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
                    #"accelerator_count": 2,
                },
            }
        ],
    )

    model_upload_op = gcc_aip.ModelUploadOp(
        project=project,
        display_name=model_display_name,
        artifact_uri=WORKING_DIR,
        serving_container_image_uri=serving_container_image_uri,
        serving_container_environment_variables={"NOT_USED": "NO_VALUE"},
    )
    model_upload_op.after(train_task)

    endpoint_create_op = gcc_aip.EndpointCreateOp(
        project=project,
        display_name="pipelines-created-endpoint",
    )

    model_deploy_op = gcc_aip.ModelDeployOp(  # noqa: F841
        project=project,
        endpoint=endpoint_create_op.outputs["endpoint"],
        model=model_upload_op.outputs["model"],
        deployed_model_display_name=model_display_name,
        machine_type="n1-standard-4",
    )

### Compile the pipeline

In [30]:
from kfp.v2 import compiler  # noqa: F811

compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path="Redbull_Classification_Pipeline.json",
)

### Run the pipeline

In [31]:
DISPLAY_NAME = "Redbull-pipeline-" + TIMESTAMP

job = aip.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="Redbull_Classification_Pipeline.json",
    pipeline_root=PIPELINE_ROOT,
)

job.run()

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/567299754976/locations/us-central1/pipelineJobs/train-endpoint-deploy2021-10-02-06-0410-20211002060804
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/567299754976/locations/us-central1/pipelineJobs/train-endpoint-deploy2021-10-02-06-0410-20211002060804')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/train-endpoint-deploy2021-10-02-06-0410-20211002060804?project=567299754976
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/567299754976/locations/us-central1/pipelineJobs/train-endpoint-deploy2021-10-02-06-0410-20211002060804 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aip

### Cleaning up

In [48]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

try:
    if delete_model and "DISPLAY_NAME" in globals():
        models = aip.Model.list(
            filter=f"display_name={DISPLAY_NAME}", order_by="create_time"
        )
        model = models[0]
        aip.Model.delete(model)
        print("Deleted model:", model)
except Exception as e:
    print(e)

try:
    if delete_endpoint and "DISPLAY_NAME" in globals():
        endpoints = aip.Endpoint.list(
            filter=f"display_name={DISPLAY_NAME}_endpoint", order_by="create_time"
        )
        endpoint = endpoints[0]
        endpoint.undeploy_all()
        aip.Endpoint.delete(endpoint.resource_name)
        print("Deleted endpoint:", endpoint)
except Exception as e:
    print(e)

if delete_dataset and "DISPLAY_NAME" in globals():
    if "tabular" == "tabular":
        try:
            datasets = aip.TabularDataset.list(
                filter=f"display_name={DISPLAY_NAME}", order_by="create_time"
            )
            dataset = datasets[0]
            aip.TabularDataset.delete(dataset.resource_name)
            print("Deleted dataset:", dataset)
        except Exception as e:
            print(e)

    if "tabular" == "image":
        try:
            datasets = aip.ImageDataset.list(
                filter=f"display_name={DISPLAY_NAME}", order_by="create_time"
            )
            dataset = datasets[0]
            aip.ImageDataset.delete(dataset.resource_name)
            print("Deleted dataset:", dataset)
        except Exception as e:
            print(e)

    if "tabular" == "text":
        try:
            datasets = aip.TextDataset.list(
                filter=f"display_name={DISPLAY_NAME}", order_by="create_time"
            )
            dataset = datasets[0]
            aip.TextDataset.delete(dataset.resource_name)
            print("Deleted dataset:", dataset)
        except Exception as e:
            print(e)

    if "tabular" == "video":
        try:
            datasets = aip.VideoDataset.list(
                filter=f"display_name={DISPLAY_NAME}", order_by="create_time"
            )
            dataset = datasets[0]
            aip.VideoDataset.delete(dataset.resource_name)
            print("Deleted dataset:", dataset)
        except Exception as e:
            print(e)

try:
    if delete_pipeline and "DISPLAY_NAME" in globals():
        pipelines = aip.PipelineJob.list(
            filter=f"display_name={DISPLAY_NAME}", order_by="create_time"
        )
        pipeline = pipelines[0]
        aip.PipelineJob.delete(pipeline.resource_name)
        print("Deleted pipeline:", pipeline)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME

list index out of range
list index out of range
list index out of range
'str' object has no attribute '_latest_future'
Removing gs://pipeline-2021-10-02-06-0410/pipeline_root/#1633154906609986...
Removing gs://pipeline-2021-10-02-06-0410/pipeline_root/redbull/#1633154907792370...
Removing gs://pipeline-2021-10-02-06-0410/pipeline_root/redbull/2021-10-02-06-0410.data-00000-of-00001#1633155342678344...
Removing gs://pipeline-2021-10-02-06-0410/pipeline_root/redbull/2021-10-02-06-0410.index#1633155343780340...
/ [4 objects]                                                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m rm ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Removing gs://pipeline-2021-10-02-06-0410/pipeline_root/redbull/2021-10-02-06-0410/#1633155174649422...
Removing gs://pipeline-2021-10-02-06-0410/pipeline